In [54]:
from sklearn.cluster import KMeans  # Cluster Algorithm calculations
import requests # Library for requests
import pandas as pd # library for data analysis
import numpy as np  # handels vector data
from  bs4 import BeautifulSoup  # scrapping the website
from IPython.display import display_html
from pandas.io.json import json_normalize # convert JSON file to Pandas dataframe
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # Address to Latitiude & Longitude
import matplotlib.cm as cm  # plotting modules
import matplotlib.colors as colors  # colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
! pip install folium  # for visual maps
import folium
print("Folium installed")
print("Libraries setup")

Folium installed
Libraries setup


In [55]:
Web_url= requests.get('https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi').text # wikipedia page on Delhi
soup=BeautifulSoup(Web_url, 'lxml') # using lxml for parsing the html
div = soup.find('div', class_="mw-parser-output" ) # delhi divisions html 
print("Web Page Imported")
#print(soup.prettify())


Web Page Imported


In [56]:
#Code to extract the relevent data from the request object using beautiful soup
 data = pd.DataFrame(columns=['Borough','Neighborhood'])
 v=-1
 flag = False  # initialize the initial event
 no = 0
 borough_temp = None  # initialize the var to NULL
# Using Childeren nodes in html to extract info
 for child in div.children:
     if child.name:
         span = child.find('span')
         if span!=-1 and span is not None:
             try:
                 if span['class'][0] == 'mw-headline' and child.a.text!='edit':
                     borough_temp = child.a.text
                     v+=1
                     flag = True
                     continue
             except KeyError:
                 continue
         if child.name=='ul' and flag==True:   # orderd list extraction
             neighborhood = []
             for ch in child.children:
                
                 try:
                     data.loc[no]=[borough_temp,ch.text]
                     no +=1
                 except AttributeError:
                     pass
         flag = False
        
 data[1:10]


,Borough,Neighborhood
1,North West Delhi,Ashok Vihar
2,North West Delhi,Azadpur
3,North West Delhi,Bawana
4,North West Delhi,Begum Pur
5,North West Delhi,Dhaka
6,North West Delhi,Jahangirpuri
7,North West Delhi,Karala
8,North West Delhi,Keshav Puram
9,North West Delhi,Kingsway Camp


Fiding the Latitude and Longitude of Each Neoghborhood

In [57]:
 ll = pd.DataFrame(columns=['latitude','longitude']) # Finding Lat and Long
 geolocator = Nominatim(user_agent="Delhi_explorer")
 for i in range(184):
     address = data['Neighborhood'].loc[i]+', New Delhi'
     try: 
         location = geolocator.geocode(address)
         ll.loc[i]=[location.latitude,location.longitude]
     except AttributeError:
         continue

In [58]:
 df1 = data
 df2 = ll
 Delhi_neighborhood_data = pd.concat([df1, df2], axis=1)
 Delhi_neighborhood_data.to_csv(r'C:\Prahlad\Desktop\FINAL_CAPSTONE_IBM\Delhi_dataSet.csv')
print("file saved")

file saved


In [59]:
delhi_neighborhood_data = pd.read_csv('C:\Prahlad\Desktop\FINAL_CAPSTONE_IBM\Delhi_dataSet.csv')
delhi_neighborhood_data.dropna(inplace=True)
delhi_neighborhood_data.reset_index(inplace=True)
delhi_neighborhood_data.drop(['index','Unnamed: 0'], axis=1, inplace=True)
delhi_neighborhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.714401,77.167288
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Dhaka,28.708698,77.205749


In [60]:
delhi_neighborhood_data.shape

(165, 4)

In [61]:
delhi_neighborhood_data.dtypes

Borough          object
Neighborhood     object
latitude        float64
longitude       float64
dtype: object

Getting rid of 'Not assigned' Boroughs preparing the Data for further Processing

Dropping Neighborhoods whse Lat and Longitude are not in Geospatial lat long of Delhi

In [62]:
delhi_neighborhood_data.info

<bound method DataFrame.info of               Borough        Neighborhood   latitude  longitude
0    North West Delhi        Adarsh Nagar  28.714401  77.167288
1    North West Delhi         Ashok Vihar  28.699453  77.184826
2    North West Delhi             Azadpur  28.707657  77.175547
3    North West Delhi              Bawana  28.799660  77.032885
4    North West Delhi               Dhaka  28.708698  77.205749
5    North West Delhi        Jahangirpuri  28.725972  77.162658
6    North West Delhi              Karala  28.735140  77.032511
7    North West Delhi        Keshav Puram  28.688926  77.161683
8    North West Delhi       Kingsway Camp  28.613749  77.212133
9    North West Delhi       Kohat Enclave  28.698041  77.140539
10   North West Delhi          Model Town  39.031714 -90.261223
11   North West Delhi              Narela  28.842610  77.091835
12   North West Delhi          Pitam Pura  28.703268  77.132250
13   North West Delhi           Rani Bagh  28.685982  77.132524
14   Nor

LAT Long Delhi

In [63]:
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="Delhi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi City are 28.6141793, 77.2022662.


Create a map of Delhi with neighborhoods superimposed on top using folium

In [64]:
Delhi_Data=delhi_neighborhood_data

In [65]:
import folium

map_Delhi = folium.Map(location=[latitude, longitude], zoom_start=11) # Map of Delhi with Neighborhoods

# add markers to map
for lat, lng, label in zip(Delhi_Data['latitude'],Delhi_Data['longitude'], Delhi_Data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
       # color='blue',
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Delhi)  
    
map_Delhi

Define Foursquare Credentials and Version

In [66]:
CLIENT_ID = '4MHLZKJMOU4XNSNNYXITLTTVAY0KWXOWEYA5KFERCNBLOZBK'
CLIENT_SECRET = '5LYM4T1ED2Q5RHBRWAGEMT2APQHPNYW3CBC5W0LU3YQK02RQ'
VERSION = '20200524'
print('Your credentails:') 
print('CLIENT_ID: ' + CLIENT_ID) 
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 4MHLZKJMOU4XNSNNYXITLTTVAY0KWXOWEYA5KFERCNBLOZBK
CLIENT_SECRET:5LYM4T1ED2Q5RHBRWAGEMT2APQHPNYW3CBC5W0LU3YQK02RQ


Let's explore the first neighborhood in our dataframe
Get the neighborhood's name.

In [67]:
First_Neighbor = Delhi_Data.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{First_Neighbor}'.")

The first neighborhood's name is 'Adarsh Nagar'.


Get the neighborhood's latitude and longitude values

In [68]:
neighborhood_latitude = Delhi_Data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = Delhi_Data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = Delhi_Data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Adarsh Nagar are 28.714400800000003, 77.1672884.


Let's get the top 100 venues that are in The Business District within a radius of 500 meters
First, let's create the GET request URL. Name your URL url

In [69]:
LIMIT=100  #The limit of venues returned by Foursquare API
radius = 500  #define the radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                                CLIENT_ID,
                                CLIENT_SECRET, 
                                VERSION, 
                                neighborhood_latitude,
                                neighborhood_longitude, 
                                radius, 
                                LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4MHLZKJMOU4XNSNNYXITLTTVAY0KWXOWEYA5KFERCNBLOZBK&client_secret=5LYM4T1ED2Q5RHBRWAGEMT2APQHPNYW3CBC5W0LU3YQK02RQ&v=20200524&ll=28.714400800000003,77.1672884&radius=500&limit=100'

Examine the resutls use the GET request and

In [70]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec601500cc1fd001b9b549d'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'North Delhi',
  'headerFullLocation': 'North Delhi, Delhi',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 28.71890080450001,
    'lng': 77.17240980502713},
   'sw': {'lat': 28.709900795499998, 'lng': 77.16216699497288}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f2ba8bfe4b0ebf428456bfd',
       'name': 'Vishyavidyalaya Metro Station@Entry gate #1 n Platform # 2(Towards Jahangir puri)',
       'location': {'address': 'North Delhi',
        'crossStreet': 'Kingsway Camp Road

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [71]:
# Extracting the category of the venue by the get_category funtion
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Convert the json file into a pandas dataframe First filter the columns then filter category for each row and finally clean the columns

In [72]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Vishyavidyalaya Metro Station@Entry gate #1 n ...,Train Station,28.715596,77.170981
1,Adarsh Nagar Metro Station,Light Rail Station,28.716598,77.170436
2,Pahalwan Dhaba,Indian Restaurant,28.714594,77.172155


In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


2. # Explore Neighborhoods in Delhi

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
    # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [81]:
Delhi_venues = getNearbyVenues(names=Delhi_Data['Neighborhood'],
                                   latitudes=Delhi_Data['latitude'],
                                   longitudes=Delhi_Data['longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Model Town
Narela
Pitam Pura
Rani Bagh
Rithala
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Pul Bangash
Roshanara Bagh
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shakti Nagar
Shastri Nagar
Tis Hazari
Timarpur
Wazirabad
Babarpur
Dilshad Garden
Kabir Nagar
Mandoli
Maujpur
Mukherjee Nagar
Nand Nagri
Naveen Shahdara
Sadatpur
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Bara Hindu Rao
Paharganj
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Laxmibai Nagar
Pragati Maidan
Brij Puri
East Vinod Nagar
Gandhi Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Preet Vihar
Anand Vihar
Shakarpur
Shastri Nagar

In [86]:
Delhi_venues = getNearbyVenues(names=Delhi_Data['Neighborhood'],
                                   latitudes=Delhi_Data['latitude'],
                                   longitudes=Delhi_Data['longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Pitam Pura
Rani Bagh
Rithala
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Roshanara Bagh
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shakti Nagar
Tis Hazari
Timarpur
Wazirabad
Babarpur
Kabir Nagar
Mandoli
Maujpur
Mukherjee Nagar
Nand Nagri
Naveen Shahdara
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Chandni Chowk
Jhandewalan
Karol Bagh
Bara Hindu Rao
Paharganj
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Laxmibai Nagar
Pragati Maidan
Brij Puri
East Vinod Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Anand Vihar
Shakarpur
Vishwas Nagar
Vivek Vihar
Shahdara
Alaknanda
Chhattarpur
Chittaranjan Park
Defence Colony
East of Kailash
Friends Colony
Greater

Let's check the size of the resulting dataframe

Load the Restauranrt List of Delhi Restaurants down loaded from zomato

In [82]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a2862d4bbd3a41b8999897c480def47b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XWzJek5RQbZ6JghxL5bmwBXaFUeOICP7vBB905hhYsJ5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_a2862d4bbd3a41b8999897c480def47b.get_object(Bucket='capstonethebattleofneighborhoods-donotdelete-pr-lmmebihshcaa4k',Key='restaurant_dataSet.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Delhi_Venues = pd.read_csv(body)
Delhi_Venues.head()


,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Adarsh Nagar,28.614193,77.071541,Eagle Boys Pizza,28.615595,77.070784,Pizza Place
1,1,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
2,2,Adarsh Nagar,28.614193,77.071541,Bikano East Patel Nagar,28.616190,77.066978,Fast Food Restaurant
3,3,Adarsh Nagar,28.614193,77.071541,McDonald's,28.616330,77.067034,Fast Food Restaurant
4,4,Ashok Vihar,28.699453,77.184826,Nat Khat Caterers,28.699630,77.187832,Indian Restaurant


In [83]:
print(Delhi_venues.shape)

(1012, 7)


In [95]:
import folium

map_Restaurants = folium.Map(location=[latitude, longitude], zoom_start=11) # Map of Delhi Restaurants

# add markers to map
for lat, lng, label in zip(Delhi_Venues['Venue Latitude'], Delhi_Venues['Venue Longitude'], Delhi_Venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius= 2,
      #  popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Restaurants)  
    
map_Restaurants

Let's check how many venues were returned for each neighborhood

In [96]:
Delhi_Venues.groupby('Neighborhood').count()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Adarsh Nagar,4,4,4,4,4,4,4
Alaknanda,9,9,9,9,9,9,9
Anand Vihar,6,6,6,6,6,6,6
Ashok Vihar,4,4,4,4,4,4,4
Azadpur,4,4,4,4,4,4,4
Bali Nagar,4,4,4,4,4,4,4
Bara Hindu Rao,8,8,8,8,8,8,8
Barakhamba Road,4,4,4,4,4,4,4
Brij Puri,2,2,2,2,2,2,2


Let's find out how many unique categories can be curated from all the returned venues

In [97]:
print('There are {} uniques categories.'.format(len(Delhi_Venues['Venue Category'].unique())))

There are 74 uniques categories.


# 3. Analyze Each Neighborhood

In [98]:
# one hot encoding
Delhi_onehot = pd.get_dummies(Delhi_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Delhi_onehot['Neighborhood'] = Delhi_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Delhi_onehot.columns[-1]] + list(Delhi_onehot.columns[:-1])
Delhi_onehot = Delhi_onehot[fixed_columns]

Delhi_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Awadhi Restaurant,BBQ Joint,Bagel Shop,Bakery,Bengali Restaurant,Bistro,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Cafeteria,Café,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Hot Dog Joint,Hyderabadi Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Irani Cafe,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Mughlai Restaurant,Multicuisine Indian Restaurant,New American Restaurant,North Indian Restaurant,Northeast Indian Restaurant,Parsi Restaurant,Pizza Place,Portuguese Restaurant,Punjabi Restaurant,Restaurant,Sandwich Place,Scandinavian Restaurant,Snack Place,Soup Place,South Indian Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [99]:
Delhi_onehot.shape

(849, 75)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [100]:
Delhi_grouped = Delhi_onehot.groupby('Neighborhood').mean().reset_index()
Delhi_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Awadhi Restaurant,BBQ Joint,Bagel Shop,Bakery,Bengali Restaurant,Bistro,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Cafeteria,Café,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Hot Dog Joint,Hyderabadi Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Irani Cafe,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Mughlai Restaurant,Multicuisine Indian Restaurant,New American Restaurant,North Indian Restaurant,Northeast Indian Restaurant,Parsi Restaurant,Pizza Place,Portuguese Restaurant,Punjabi Restaurant,Restaurant,Sandwich Place,Scandinavian Restaurant,Snack Place,Soup Place,South Indian Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Adarsh Nagar,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Alaknanda,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.222222,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Anand Vihar,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ashok Vihar,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Azadpur,0.000000,0.000000,0.25,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,

Let's confirm the new size

In [101]:
Delhi_grouped.shape

(116, 75)

In [102]:
for i in Delhi_grouped.columns:
    print(i,end=", ")

Neighborhood, Afghan Restaurant, American Restaurant, Argentinian Restaurant, Asian Restaurant, Australian Restaurant, Awadhi Restaurant, BBQ Joint, Bagel Shop, Bakery, Bengali Restaurant, Bistro, Breakfast Spot, Buffet, Burger Joint, Burmese Restaurant, Cafeteria, Café, Chinese Restaurant, Comfort Food Restaurant, Deli / Bodega, Dhaba, Dim Sum Restaurant, Diner, Doner Restaurant, Donut Shop, Dumpling Restaurant, Eastern European Restaurant, Falafel Restaurant, Fast Food Restaurant, Fish & Chips Shop, Food, Food Court, Food Stand, Food Truck, French Restaurant, Fried Chicken Joint, Gastropub, Hot Dog Joint, Hyderabadi Restaurant, Indian Chinese Restaurant, Indian Restaurant, Indian Sweet Shop, Irani Cafe, Italian Restaurant, Japanese Restaurant, Korean Restaurant, Mediterranean Restaurant, Mexican Restaurant, Middle Eastern Restaurant, Modern European Restaurant, Molecular Gastronomy Restaurant, Mughlai Restaurant, Multicuisine Indian Restaurant, New American Restaurant, North Indian R

In [113]:
num_top_venues = 5

for hood in Delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Delhi_grouped[Delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Nagar----
                     venue  freq
0     Fast Food Restaurant  0.50
1        Indian Restaurant  0.25
2              Pizza Place  0.25
3       Mexican Restaurant  0.00
4  New American Restaurant  0.00


----Alaknanda----
                       venue  freq
0          Indian Restaurant  0.22
1                  BBQ Joint  0.22
2                Pizza Place  0.11
3  Middle Eastern Restaurant  0.11
4    New American Restaurant  0.11


----Anand Vihar----
                venue  freq
0   Indian Restaurant  0.33
1          Soup Place  0.17
2   Indian Sweet Shop  0.17
3         Pizza Place  0.17
4  Punjabi Restaurant  0.17


----Ashok Vihar----
                       venue  freq
0          Indian Restaurant  0.50
1                      Diner  0.25
2                     Bakery  0.25
3          Afghan Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Azadpur----
                    venue  freq
0  Argentinian Restaurant  0.25
1       Indian Restaurant  0.25
2              

                     venue  freq
0        Indian Restaurant  0.50
1        Fish & Chips Shop  0.25
2                     Café  0.25
3       Mexican Restaurant  0.00
4  New American Restaurant  0.00


----Jhilmil Colony----
                       venue  freq
0          Indian Restaurant  0.67
1           Asian Restaurant  0.33
2          Afghan Restaurant  0.00
3  Middle Eastern Restaurant  0.00
4    North Indian Restaurant  0.00


----Jor Bagh----
                     venue  freq
0                     Café   0.5
1         Sushi Restaurant   0.5
2        Afghan Restaurant   0.0
3       Mexican Restaurant   0.0
4  New American Restaurant   0.0


----Kailash Colony----
                venue  freq
0   Indian Restaurant  0.29
1   Afghan Restaurant  0.14
2              Bakery  0.14
3  Italian Restaurant  0.14
4         Snack Place  0.14


----Kamla Nagar----
                 venue  freq
0    Indian Restaurant  0.27
1       Breakfast Spot  0.27
2  Fried Chicken Joint  0.09
3           Donut S

                     venue  freq
0     Fast Food Restaurant  0.50
1       Chinese Restaurant  0.25
2               Food Court  0.25
3        Afghan Restaurant  0.00
4  New American Restaurant  0.00


----Pandav Nagar----
                       venue  freq
0          Indian Restaurant  0.67
1                       Café  0.33
2          Afghan Restaurant  0.00
3  Middle Eastern Restaurant  0.00
4    North Indian Restaurant  0.00


----Paschim Vihar----
                       venue  freq
0                       Café   0.5
1                  BBQ Joint   0.5
2          Afghan Restaurant   0.0
3  Middle Eastern Restaurant   0.0
4    North Indian Restaurant   0.0


----Patel Nagar----
                       venue  freq
0                       Café   1.0
1          Afghan Restaurant   0.0
2  Middle Eastern Restaurant   0.0
3    North Indian Restaurant   0.0
4    New American Restaurant   0.0


----Patparganj----
                     venue  freq
0        Indian Restaurant   0.6
1               

4  Multicuisine Indian Restaurant   0.0


----Wazirabad----
                            venue  freq
0                     Snack Place   1.0
1               Afghan Restaurant   0.0
2              Mexican Restaurant   0.0
3         New American Restaurant   0.0
4  Multicuisine Indian Restaurant   0.0


----Yamuna Vihar----
                            venue  freq
0         North Indian Restaurant   0.5
1             Dumpling Restaurant   0.5
2     Northeast Indian Restaurant   0.0
3         New American Restaurant   0.0
4  Multicuisine Indian Restaurant   0.0




In [103]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's print each neighborhood along with the top 5 most common venues

In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
        # create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Delhi_grouped['Neighborhood']

for ind in np.arange(Delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
1,Alaknanda,BBQ Joint,Indian Restaurant,New American Restaurant,Restaurant,Middle Eastern Restaurant,Pizza Place,Steakhouse,Deli / Bodega,Dhaba,Dim Sum Restaurant
2,Anand Vihar,Indian Restaurant,Pizza Place,Indian Sweet Shop,Soup Place,Punjabi Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Deli / Bodega,Dhaba,Dim Sum Restaurant
3,Ashok Vihar,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,Azadpur,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


Let's put that into a pandas dataframe

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

#  4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [124]:
# set number of clusters
kclusters = 5

Delhi_grouped_clustering = Delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 2, 3, 1, 3, 3, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [126]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

Delhi_merged = Delhi_Data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Delhi_merged = Delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Delhi_merged.dropna(inplace=True)
Delhi_merged.head() # check the last columns!

,Borough,Neighborhood,latitude,longitude,Cluster_Labels,Cluster labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North West Delhi,Adarsh Nagar,28.714401,77.167288,0.0,0.0,0.0,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
1,North West Delhi,Ashok Vihar,28.699453,77.184826,2.0,2.0,2.0,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
2,North West Delhi,Azadpur,28.707657,77.175547,3.0,3.0,3.0,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
7,North West Delhi,Keshav Puram,28.688926,77.161683,3.0,3.0,3.0,Gastropub,Indian Restaurant,Café,Bakery,Food Truck,Food Stand,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant
9,North West Delhi,Kohat Enclave,28.698041,77.140539,2.0,2.0,2.0,Indian Restaurant,Bakery,Food Court,Food,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


Finally, let's visualize the resulting clusters

In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Delhi_merged['latitude'], Delhi_merged['longitude'], Delhi_merged['Neighborhood'], Delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Analyze Each Cluster Data

In [134]:
clusterdata = pd.merge(Delhi_onehot.groupby('Neighborhood').sum(),Delhi_merged[['Neighborhood','Cluster Labels']],left_on='Neighborhood', right_on='Neighborhood',how='inner')
clusterdata = clusterdata.iloc[:,1:].groupby('Cluster Labels').sum().transpose()
clusterdata.head()

Cluster Labels,0.0,1.0,2.0,3.0,4.0
Afghan Restaurant,0,0,0,10,0
American Restaurant,1,0,1,11,0
Argentinian Restaurant,0,0,0,1,0
Asian Restaurant,1,0,2,18,0
Australian Restaurant,0,0,0,1,0


In [ ]:
import seaborn as sns

def plot_bar(clusternumber):
    sns.set(style="whitegrid",rc={'figure.figsize':(20,10)})
    df = clusterdata[[clusternumber]].drop(clusterdata[[clusternumber]][clusterdata[clusternumber]==0].index)
    chart = sns.barplot(x=df.index, y=clusternumber, data=df)
    chart.set_xticklabels(chart.get_xticklabels(),rotation=90)
 

In [ ]:
   plot_bar(0)

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x="day", y="total_bill", data=tips)